In [5]:
import subprocess

def run_chess_test(opening_moves, timeout=240):
    # Join the moves into a single string without list formatting
    opening_moves_str = " ".join(str(move) for move in opening_moves)

    # Define the command and options for running the Robot Framework script
    robot_command = [
        "robot",
        "--variable", f"OPENING_MOVES:{opening_moves_str}",
        "chessML.robot"
    ]
    
    print("Running command:", robot_command)
    # Start the Robot Framework process
    process = subprocess.Popen(robot_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    try:
        # Capture and print the output in real-time, with a timeout
        stdout, stderr = process.communicate(timeout=timeout)

        print(stdout) 
        if stderr:
            print(stderr)

    except subprocess.TimeoutExpired:
        print("Process timed out; forcing termination.")
        process.kill()  # Forcefully terminate the process after timeout
        stdout, stderr = process.communicate()
        print(stdout)
        if stderr:
            print(stderr)

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        process.terminate()

    print("Process finished and resources cleaned up.\n")

#list of opening moves to test
opening_moves = [["e2e4", "g1f3", "f1c4"], ["e2e4", "e7e5", "f1c4"]]

run_chess_test(opening_moves[0])

Running command: ['robot', '--variable', 'OPENING_MOVES:e2e4 g1f3 f1c4', 'chessML.robot']
Process timed out; forcing termination.
chessML                                                                       
Test PlayOpening                                                      | PASS |
------------------------------------------------------------------------------
chessML                                                               | PASS |
1 test, 1 passed, 0 failed
Output:  /Users/marcokaferbeck/DataspellProjects/roboticProccessAutomationChess/output.xml
Log:     /Users/marcokaferbeck/DataspellProjects/roboticProccessAutomationChess/log.html
Report:  /Users/marcokaferbeck/DataspellProjects/roboticProccessAutomationChess/report.html

Process finished and resources cleaned up.



In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv('game_results.csv')
X = data[['move1', 'move2', 'move3', 'bot1', 'bot2']]
y = data['Result']

# Update y to replace any text inside parentheses with an empty string, only for "Draw" results
y = y.str.replace(r"\(.*\)", "", regex=True).str.strip()

# Convert categorical data to numeric for ML
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train the RandomForestClassifier on the training set
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model on the training set
train_score = model.score(X_train, y_train)

# Evaluate the model on the testing set
test_score = model.score(X_test, y_test)

# Print the training and testing scores
print("Training score:", train_score)
print("Testing score:", test_score)

# Create a sample input for prediction
sample_input = pd.DataFrame({
    'move1': ['e2e4'],
    'move2': ['e7e5'],
    'move3': ['g1f3'],
    'bot1': ['Stockfish'],
    'bot2': ['LCZero']
})

# Convert the sample input to the same format as the training data
sample_input = pd.get_dummies(sample_input)
sample_input = sample_input.reindex(columns=X.columns, fill_value=0)

# Predict the result for the sample input
prediction = model.predict(sample_input)

# Print the prediction result
print("Prediction for sample input:", prediction[0])

Training score: 0.8833333333333333
Testing score: 0.75
Prediction for sample input: White wins!
